# Q learning - Non-linear Regression Attempt
In Q learning, the update rule is:
$$Q(\mathbf{s},\mathbf{a}) = Q(\mathbf{s},\mathbf{a}) + \alpha (r+\gamma max_{\mathbf{a}'} Q(\mathbf{s}',\mathbf{a}') - Q(\mathbf{s},\mathbf{a}))$$
If the Q function reached the best, there would be nothing more to learn. Hence, we wanted to make $r+\gamma max_{\mathbf{a}'} Q(\mathbf{s}',\mathbf{a}') - Q(\mathbf{s},\mathbf{a})$ close to 0. Then the problem became fitting the dataset: [$r+\gamma max_{\mathbf{a}'} Q(\mathbf{s}',\mathbf{a}')$, $Q(\mathbf{s},\mathbf{a})$]. In every step of environment evolution, the [$r+\gamma max_{\mathbf{a}'} Q(\mathbf{s}',\mathbf{a}')$, $Q(\mathbf{s},\mathbf{a})$] pair can form a dataset for fitting.
When using linear attempt, the problem is that the maximum Q is always at the bound of the action space, and the maximum value does not depend on state $\mathbf{s}$. To avoid these, we wanted a Q function with non-zero second order derivative, and the simplest one is quadratic function.
$$Q(s,a)=\sum_{i\leq j}a_{ij}\mathbf{V}_i\mathbf{V}_j, \mathbf{V}=(\mathbf{s},\mathbf{a})$$
or
$$Q(s,a)=\sum_{i,j}b_{ij}\mathbf{V}_i\mathbf{V}_j, \mathbf{V}=(\mathbf{s},\mathbf{a})\ and\  b_{ij}=b_{ji}$$

In [1]:
from osim.env import L2RunEnv
import numpy as np
from scipy.optimize import minimize, Bounds
from sklearn.linear_model import LinearRegression

DEFAULT_SEED = 20180101
rng = np.random.RandomState(DEFAULT_SEED)

env = L2RunEnv(visualize=False)
# Obtain the dimension observation space and action space
dim_obs = env.get_observation_space_size()
dim_act = env.get_action_space_size()

# Set the range of action values
action_low = -0.1
action_high = 0.1

# Set hyperparameters
discount = 0.0001
epsilon = 0.9
episode = 2000

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


## Q function
Q function in this section is the linear combination of productions of each states and action components, since we want the state can affect the maximum of action: $\frac{\partial^2Q}{\partial s_i \partial a_j}\neq 0$. We stored the coefficients of these quadratic terms and updated them during linear fitting, which fit the data pair $[\{V_iV_j\}, Q]$, where $\mathbf{V}=(\mathbf{s}, \mathbf{a})$.

In [9]:
class qfunction:
    # random initialization
    def __init__(self, dim_obs, dim_act, rng=None):
        if rng is None:
            rng = np.random.RandomState(DEFAULT_SEED)
        self.rng = rng
        self.dim = (dim_obs + dim_act + 1) * (dim_obs + dim_act) // 2
        self.coeff = rng.uniform(-1, 1, self.dim)

    def __call__(self, obs, act):
        con_vec = np.concatenate((obs,act))
        quad_vec = quadartic(con_vec)
        res = np.sum(quad_vec*self.coeff)
        return res

    def update(self, coeff):
        self.coeff = coeff
        
def quadartic(vec):
    # covert (x1, x2, x3, ...) to (x1^2, x1x2, x1x3, ..., x2^2, x2x3, ...)
    if not isinstance(vec, np.ndarray):
        vec = np.array(vec)
    res = []
    for i, x in enumerate(vec):
        res = np.concatenate((res, x*vec[i:]))
    return res

In [10]:
# Initialize Q function
qf = qfunction(dim_obs, dim_act)
model = LinearRegression(fit_intercept=False)
# Initialize the dataset:(xdata, ydata)
xdata = np.zeros((qf.dim,))
ydata = np.zeros((1,))

In [12]:
action0 = np.zeros(dim_act)
for i in range(episode):
    # Initialize a new simulation
    state = np.array(env.reset())
    reward = 0

    # Run the simulation until the framework stop it
    done = False
    while not done:
        # get the action based on Q function and epsilon greedy
        if (rng.rand() < epsilon) :
            # exploration: randomly choose an action
            action = rng.uniform(action_low, action_high, dim_act)
        else:
            # exploitation: choose the action maximumizing Q function
            action_func = lambda x: -qf(state, x)
            bnds = Bounds(action_low, action_high)
            res = minimize(action_func, action0, method='SLSQP', bounds=bnds)
            action = res.x

        # evolve the system to the next time stamp
        state_, reward, done, info = env.step(action)
        state_ = np.array(state_)

        # build the dataset
        action_func = lambda x: -qf(state_, x)
        bnds = Bounds(action_low, action_high)
        res = minimize(action_func, action0, method='SLSQP', bounds=bnds)
        max_q = -res.fun
        # {s, a} and [r + gamma * max_a` Q(s`, a`)]
        xx = quadartic(np.concatenate((state, action)))
        yy = np.array(reward + discount * max_q)
        # put the data point into dataset
        xdata = np.vstack((xdata, xx))
        ydata = np.vstack((ydata, yy))

        # Do linear regression and update Q function
        model.fit(xdata, ydata)
        qf.update(model.coef_.T)

        # Update state
        state = state_

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').